In [1]:
from scrape_tinystories import TinyStoriesParser
import random

parser = TinyStoriesParser()
stories = parser.get_stories()

print(len(stories))

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/accounts/projects/jsteinhardt/akommula/.local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


2119719


In [7]:
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM, GPTQConfig
from sentence_transformers import SentenceTransformer
import transformers

from torch.utils.data import Dataset
import torch
import torch.nn.functional as F

prefix_q = """
I will provide you with a short story. Please write a list of the protagonists of the story,
along with a list of the antagonists. Make sure all characters belong to one or the other.
Write your answer in the format: [The correct answer is: Protagonists: ..., Antagonists: ...].
"""

class InteractLLaMA:
	def __init__(self):
		self.model, self.tokenizer = self.load_llama_model()
			
	def generate_message(self, question):
		question = (prefix_q + question)
		formatted_question = question.replace("'", '')
		return ([{'role': 'system', 'content': ''}, 
				 {'role': 'user', 'content': (formatted_question)}])

	def messages_to_prompt(self, messages):
		assert len(messages) == 2
		assert messages[0]['role'] == 'system'
		assert messages[1]['role'] == 'user'
		sys_message = f"<s>[INST] <<SYS>>\n{messages[0]['content']}\n<</SYS>>\n\n"
		ins_message= f"{messages[1]['content']} [/INST]"
		prompt = sys_message + ins_message
		return prompt

	def load_llama_model(self):
		model_name_or_path = "/scratch/users/erjones/models/postprocessed_models/7B-chat"
		config = AutoConfig.from_pretrained(model_name_or_path)
		use_fast_tokenizer = "LlamaForCausalLM" not in config.architectures
		tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left")
		tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
		tokenizer.bos_token_id = 1
        
		gptq_config = GPTQConfig(bits=4, dataset="c4", tokenizer = tokenizer)      
		model = AutoModelForCausalLM.from_pretrained(model_name_or_path, quantization_config=gptq_config, device_map="auto")
		return model, tokenizer

	def extract_answer(self, llm_response, target):
		segments = llm_response.replace("[/INST]", "").split("[The correct answer is: ")
		return (segments[-1])

	def	answer_batched_questions(self, batched_questions):
		modified_questions = [self.messages_to_prompt(self.generate_message(q)) for q in batched_questions]
		
		answers = []
		batch_size = 96
		idx = 0

		pipeline = transformers.pipeline(
    		"text-generation",
    		model=self.model,
    		tokenizer=self.tokenizer,
    		torch_dtype = torch.float16, 
    		device_map = "auto"
		)

		for out in tqdm(pipeline(modified_questions, do_sample = True, top_k = 10, num_return_sequences = 1, 
        		                eos_token_id = self.tokenizer.eos_token_id, max_length = 1000, batch_size = batch_size)):

			for sequence in out:
				answer = self.extract_answer(sequence['generated_text'])
				answers.append(answer)    

		return (answers)

In [8]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

torch.cuda.empty_cache()

In [11]:
import random
random.shuffle(stories)

short_stories = stories[:50]

In [ ]:
interacter = InteractLLaMA()
answers = interacter.answer_batched_questions(short_stories)

print(answers[:100])

'''
erroneous = []
story_id = 0
for answer in answers:
    protagonists = 
    antagonists = 
    
    if bool(set(a) & set(b)):
        print("Erroneous Overlapping. story_id:", story_id)
        erroneous.append(story_id)
              
    story_id += 1

print("# errors:", len(erroneous))
    
for err in erroneous:
    print(short_stories[err])
    print('=======================')
'''

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
/accounts/projects/jsteinhardt/akommula/.local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]